
# Graffiti Annual  Numbers

# Make Sure the modules are installed



# Import modules

In [91]:

import numpy as np
import pandas as pd




In [92]:
!pwd

/home/gdillon/JupyterPractice/graffiti_SF311


# produce viz of annual total 2009-2016
because I'm using SQL type queery with count(*), this responds faster

## api call for count of all by year

In [93]:
url = 'https://data.sfgov.org/resource/qer8-n8u9.csv?$select=date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year'

In [94]:
df = pd.read_csv(url)
df = df[1:-1]
df

,year,count
1,01/01/2009 12:00:00 AM,32285
2,01/01/2010 12:00:00 AM,26769
3,01/01/2011 12:00:00 AM,25445
4,01/01/2012 12:00:00 AM,26254
5,01/01/2013 12:00:00 AM,35831
6,01/01/2014 12:00:00 AM,46808
7,01/01/2015 12:00:00 AM,57249
8,01/01/2016 12:00:00 AM,72392


In [95]:
df['year']=df['year'].str.extract('(?P<year>\d{4})').astype(str)
df['count']=df['count'].astype(int)
df

/home/gdillon/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


,year,count
1,2009,32285
2,2010,26769
3,2011,25445
4,2012,26254
5,2013,35831
6,2014,46808
7,2015,57249
8,2016,72392


In [96]:
from bokeh.charts import Bar, output_file, output_notebook
from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [97]:
trend = Bar(df, values = 'count', label = 'year', title = 'Graffiti reported to SF311 by year')
output_notebook()
show(row(trend))

Loading BokehJS ...

In [167]:
df = df[3:-1]
df

,Source,year,count
21,Web Self Service,2013,11330
22,Twitter,2013,1512
23,NaN,2014,5
24,Open311,2014,19813
25,Twitter,2014,2227
26,Web Self Service,2014,11769
27,Integrated Agency,2014,5493
28,Voice In,2014,7501
29,Other Department,2015,6
30,Open311,2015,38732


In [169]:
df.index

RangeIndex(start=21, stop=39, step=1)

In [99]:
trend = Bar(df, values = 'count', label = 'year')
show(row(trend))

# Analysis by Source, grab new dataframe by constructing SQL query to API

In [221]:
url = 'https://data.sfgov.org/resource/qer8-n8u9.csv?$select=source,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,source'

In [222]:
print(url)

https://data.sfgov.org/resource/qer8-n8u9.csv?$select=source,date_trunc_y(opened)as%20year,%20count(*)&$order=year&$group=year,source


In [223]:
df=pd.read_csv(url)
df=df[18:]
df['year']=df['year'].str.extract('(?P<year>\d{4})').astype(str)
df['count']=df['count'].astype(int)
df=df[:-1]
#df.head(3)
df

/home/gdillon/.local/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


,Source,year,count
18,Open311,2013,4030
19,Integrated Agency,2013,9666
20,Voice In,2013,9293
21,Web Self Service,2013,11330
22,Twitter,2013,1512
23,NaN,2014,5
24,Open311,2014,19813
25,Twitter,2014,2227
26,Web Self Service,2014,11769
27,Integrated Agency,2014,5493


In [224]:
df2 =pd.pivot_table(
  df,values='count',
  index=['year'], 
  columns=['Source'],
  aggfunc=np.sum) 
df2

Source,Integrated Agency,Open311,Other Department,Twitter,Voice In,Web Self Service
year,,,,,,
2013,9666.0,4030.0,NaN,1512.0,9293.0,11330.0
2014,5493.0,19813.0,NaN,2227.0,7501.0,11769.0
2015,8530.0,38732.0,6.0,202.0,5636.0,4143.0
2016,7992.0,55148.0,NaN,194.0,4390.0,4668.0


In [225]:

df2.drop(df2.columns[[2]], axis=1, inplace=True)
df2

Source,Integrated Agency,Open311,Twitter,Voice In,Web Self Service
year,,,,,
2013,9666.0,4030.0,1512.0,9293.0,11330.0
2014,5493.0,19813.0,2227.0,7501.0,11769.0
2015,8530.0,38732.0,202.0,5636.0,4143.0
2016,7992.0,55148.0,194.0,4390.0,4668.0


In [226]:
df2.index

Index(['2013', '2014', '2015', '2016'], dtype='object', name='year')

example code from stackoverflow http://stackoverflow.com/questions/31520951/plotting-multiple-lines-with-bokeh-and-pandas

In [228]:
import numpy as np
from bokeh.charts import Line, output_file, show

# (dict, OrderedDict, lists, arrays and DataFrames are valid inputs)
#xyvalues = np.array([[2, 3, 7, 5, 26], [12, 33, 47, 15, 126], [22, 43, 10, 25, 26]])
xyvalues = df2
line = Line(xyvalues, title="line", legend="top_left", ylabel='Reports to SF311 by Year')

output_file('line.html')
show(line)

INFO:bokeh.core.state:Session output file 'line.html' already exists, will be overwritten.


In [110]:
from bokeh.palettes import Spectral11
from bokeh.charts import Line, show, output_file, output_notebook


line1 = Line(df2, title="Line Chart", legend="top_left",
             xlabel='Source', ylabel='count')




output_notebook()
show(p)

ValueError: expected an element of either Column Name or Column String or List(Column Name or Column String), got [('count', 'Integrated Agency'), ('count', 'Open311'), ('count', 'Twitter'), ('count', 'Voice In'), ('count', 'Web Self Service')]

# The above graph needs labels for years and what the line represent.   But it shows that all the growth in reports is coming from the mobile app.

# Grab url and begin pulling year of data

In [ ]:
url = 'https://data.sfgov.org/resource/rme2-xgvs.json'
url = url[:-4]+'csv'
url

# Growth in reports (2013 to 2016)

### Code for dealing with Socrata limit of 50,000 api limit

In [ ]:

limit = '?$limit=50000'
offset = '&$offset=50000'
url1 = url+limit
print (url1)

url2 = url1 + offset
print (url2)

In [ ]:
# drop from url those prior to January 1
where = '&$where='
category = 'opened'
start = '>='

#date format 'year-mm-dd'
onorafter = "'2016-01-01'"
url2 = url2 + where + category + start + str(onorafter) 
print(url2)

# Concatenate or Vertically stack the two dataFrames

In [ ]:
df1  = pd.read_csv(url1)
df2 = pd.read_csv(url2)
df = pd.concat([df1,df2])
df.shape


In [ ]:
# frames = [df1,df2]
# df = pd.concat(frames)

# Convert the time series into pandas time version

In [ ]:
df.columns

In [ ]:
type(df.Opened)

In [ ]:
df['Opened'] = pd.to_datetime(df['Opened']).dt.strftime('%Y-%m-%d')

In [ ]:
df['Closed'] = pd.to_datetime(df['Closed']).dt.strftime('%Y-%m-%d')
df['Updated'] = pd.to_datetime(df['Updated']).dt.strftime('%Y-%m-%d')


In [ ]:
df.head(2)

In [ ]:
df.to_string(columns='Supervisor District')

In [ ]:
df.to_string(columns = 'Supervisor District')

In [ ]:
df['Diff'] =(df.Closed-df.Opened).astype('timedelta64[D]')


In [ ]:
type(df['Supervisor District'])

In [ ]:
df.Diff.describe()

In [ ]:
df[df['Neighborhood' ]== "Mission"].head(85)

In [ ]:
df[df['Neighborhood' ]== "Mission"].describe()

In [ ]:
df['Responsible Agency'].value_counts()[:6]

In [ ]:
df[df['Responsible Agency'] == 'US Postal Service Maintenance Queue']['Diff'].describe()

In [ ]:
df[df['Responsible Agency'] == 'Clear Channel - Transit Queue']['Diff'].describe()

In [ ]:
df[df['Responsible Agency'] == 'DPW Ops Queue']['Diff'].describe()

In [ ]:
df[df['Category'] == 'Graffiti Private Property']['Diff'].describe()

In [ ]:
df[df['Category'] == 'Graffiti Public Property']['Diff'].describe()

In [ ]:
df['Source'].value_counts()

In [ ]:
df[df['Source'] == 'Open311']['Diff'].describe()

In [ ]:
df[df['Source'] == 'Voice In']['Diff'].describe()

In [ ]:
df[df['Source'] == 'Integrated Agency']['Diff'].describe()

# Consider some exploratory statistics / reality checks

In [ ]:
source = df.Source.value_counts()
print(type(source))


print(source)

In [ ]:
type(source)

In [ ]:
label_list = source.index.tolist()
print(label_list)

In [ ]:
source = source.tolist()
source


In [ ]:


plt.axis("equal")
plt.pie(
        source,
        labels=label_list,
        autopct="%1.1f%%"
        )
plt.title("Graffiti Reports Submitted in 2016")
plt.show()

In [ ]:
sups = df['Supervisor District'].value_counts().head(11)

In [ ]:
top = 

# Get list of Supervisor's'

In [ ]:
df['Status Notes'].value_counts().head(10)

In [ ]:
df['Status'].value_counts().head()

In [ ]:
df['Responsible Agency'].value_counts().head(8)

In [ ]:
df.Category.value_counts().head()

In [ ]:
df['Request Type'].value_counts().head(10)

In [ ]:
df['Request Details'].value_counts().head(20)

In [ ]:
df['Address'].value_counts().head(20)

In [ ]:
df['Source'].value_counts().head(6)

In [ ]:
df['Media URL'].describe()

In [ ]:
df['Point'].describe()

# Plot this.    Simple First


In [ ]:
#Plot by top responsible agency that is not DPW

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!cat geocoded.csv

# Start Plotting the simple items

# Plot Responsible Agency non DPW

In [ ]:
respAgencyCount = df['Responsible Agency'].value_counts()

In [ ]:
respAgencyCount = respAgencyCount[1:27]

In [ ]:
respAgencyCount

## minimally acceptable is to do pie / dpw vs all other
### DPW share compared to prior years
### Conclude, DPW doing a good job, how about the other departments
### Bar Chart the top 5-6
### Which of the top 5-6 are doing better.
### can't give the most natural statistic, because 